In [2]:
import pandas as pd
import pandas as pd
import re
import string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from nltk.corpus import stopwords
from string import digits
import numpy as np

In [ ]:
import nltk
from nltk.stem.porter import PorterStemmer
porter_stemmer = PorterStemmer()
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

In [ ]:
def preprocess_data(s):
    s = s.lower()
    data = re.sub(r'[^\x00-\x7F]+', ' ', s)
    final_str = data.translate(None, string.punctuation)
    filter_str = final_str.translate(None, digits)
    nltk_tokens = nltk.word_tokenize(filter_str)
    #Next find the roots of the word
    str_= ''
    for w in nltk_tokens:

        if w not in stopwords.words('english'):
            str_ += ' '  + (lemmatizer.lemmatize(w))
    
    return str_.strip()

In [ ]:
df_train = pd.read_csv('/home/prashant/Downloads/c3cc8568-0-dataset/train.csv')
df_train['tag'] = 'train'
print df_train.shape

In [ ]:
df =  df_train
df_train.head()

In [ ]:
#df_train.isnull().sum()
df['Complaint-reason'] = df['Complaint-reason'].apply(preprocess_data)

In [ ]:
#df_train.isnull().sum()
df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].apply(preprocess_data)

In [ ]:
df.head()

In [ ]:
import re
df['Consumer-complaint-summary'] = df['Consumer-complaint-summary'].map(lambda x : re.sub("x+","",x))

In [ ]:
df["merge"] = df["Consumer-complaint-summary"].map(str)+ " " + df["Complaint-reason"]


df.head()

In [3]:
import pandas as pd
df = pd.read_csv('/home/prashant/preprocess_brain.csv')

In [12]:
def get_vector(model,all_data):
    vector_sen = []
    for d in all_data:
        single_sen_vec = []
        words = d.split(' ')
        for w in words:
            try:
                
                get_word_vec = model[w]
            except:
                get_word_vec =np.zeros(300)
                pass
            single_sen_vec.append(get_word_vec)
        v = np.array(single_sen_vec).mean(axis=0)
        vector_sen.append(v)
    return vector_sen

In [4]:
def loadGloveModel(gloveFile):
    print "Loading Glove Model"
    f = open(gloveFile,'r')
    model = {}
    for line in f:
        splitLine = line.split()
        word = splitLine[0]
        embedding = np.array([float(val) for val in splitLine[1:]])
        model[word] = embedding
    print "Done.",len(model)," words loaded!"
    return model

model = loadGloveModel('/home/prashant/Downloads/kagsa/data/glove.840B.300d.txt')
#vocab = model.keys()

Loading Glove Model
Done. 2196016  words loaded!


In [13]:
# text = list(df['merge'])
# vectorizer = TfidfVectorizer(min_df=15,max_df=100)
# # tokenize and build vocab
# vectorizer.fit(text)
vector = get_vector(model,list(df['merge']))

In [11]:
# # encode document
# vector = vectorizer.transform(text)
# # summarize encoded vector
# print(vector.shape)

0        seterus inc po un fau rapport aupr de principa...
1           la requ te en faillite n  du chapitre  po e...
2        el   estaba preparando el vuelo de regreso  ve...
3        loan paid    year moved va moved     month pay...
4        jai obtenu un compte de cr dit de soins pour  ...
5        owner original mortgage filed bankruptcy later...
6        jai victime dune fraude didentit et jai contac...
7        je suis en train de faire faillite et par con ...
8        una agencia de cobranza hizo adulterar de que ...
9        le   jai u une r ponse deperian indiquant que ...
10       mortgage provider united wholesale mortgage ho...
11       boyfriend bought  sofa year pay without intere...
12       le    environ  heures j ai v rifi mon dossier ...
13       account settled closed must balance zero credi...
14          open lease  dba  kia    first person lease ...
15       scheduled full time payment online banking epi...
16       llamado esta compa muchas veces para intentar .

In [14]:
vec_df = pd.DataFrame(vector)

In [15]:
vec_df['tag'] = list(df['tag'])
vec_df['Complaint-Status'] = list(df['Complaint-Status'])
#vec_df['Transaction-Type'] = list(df['Transaction-Type'])
vec_df['Complaint-ID'] = list(df['Complaint-ID'])

In [16]:
y_train = (vec_df['Complaint-Status'])
x_train = vec_df.drop(['Complaint-Status','Complaint-ID','tag'],axis =1)

In [17]:
print y_train.shape
y_train_one_hot = pd.get_dummies(y_train,prefix=['Complaint-Status'])
print y_train_one_hot.shape

(43266,)
(43266, 5)


In [ ]:
#df.to_csv('/home/prashant/preprocess_brain.csv',index=False)

In [ ]:
print x_train.shape
print y_train.shape

In [19]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB,GaussianNB
from sklearn.linear_model import LogisticRegression
X_train, X_test, y_train_internal, y_test_internal = train_test_split(x_train,y_train ,random_state = 0)

In [20]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier

# Instantiate model with 1000 decision trees
rf = RandomForestClassifier(n_estimators = 100, random_state = 42)

# Train the model on training data
rm = rf.fit(X_train, y_train_internal)
pred_internal = rm.predict(X_test)



In [22]:
import sklearn
sklearn.metrics.f1_score(pred_internal,y_test_internal,average='weighted')

0.8805785281135934